In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# QnA with BigQuery using langchain & Vertex PALM API


This notebook guides you on how to use of LLM to answer questions over a BigQuery table.
You will access Vertex PALM API to generate SQL query and execute it agains BigQuery Tables.


### Install latest Vertex LLM SDK

In [ ]:
# Authenticate with Google account
# run the following lines ONLY if you use Google Colab

#from google.colab import auth as google_auth
#google_auth.authenticate_user()

In [1]:
#install required libraries

!pip3 uninstall -y google-cloud-aiplatform
!pip install google-cloud-aiplatform --upgrade
!pip install langchain langchain-experimental
!pip install google-cloud-core

# Install SQLAlchemy for database access to BigQuery
!pip install SQLAlchemy --quiet
!pip install sqlalchemy-bigquery

Found existing installation: google-cloud-aiplatform 1.36.4
Uninstalling google-cloud-aiplatform-1.36.4:
  Successfully uninstalled google-cloud-aiplatform-1.36.4
  Obtaining dependency information for google-cloud-aiplatform from https://files.pythonhosted.org/packages/f9/00/13c8a1c052d8205875c888281a34ea10a181d5ca7b6d2003c28fb1da1a03/google_cloud_aiplatform-1.36.4-py2.py3-none-any.whl.metadata
  Using cached google_cloud_aiplatform-1.36.4-py2.py3-none-any.whl.metadata (27 kB)
Using cached google_cloud_aiplatform-1.36.4-py2.py3-none-any.whl (3.3 MB)


In [2]:
#restart kernel
exit()

### Initiating BigQuery Parameter & LLM

In [1]:
# Change to your project ID!

project_id = "<your_project_id>" # @param {type:"string"}

location = "us-central1"  # @param {type:"string"}
dataset_id = 'clinic_dataset' # @param {type:"string"}
table_name1 = 'customerdata' # @param {type:"string"}
table_name2 = 'ordertransaction' # @param {type:"string"}
table_name3 = 'satisfactionsurvey' # @param {type:"string"}

table_names = (table_name1,table_name2,table_name3)
# table_names = (table_name1)

In [3]:
# Initialize LangChain and Vertex AI PALM API

import pandas as pd
from io import StringIO
import langchain
import vertexai
vertexai.init(project=project_id, location=location)
from langchain.llms import VertexAI


# Text model instance integrated with langChain
#create instance of LLM model using Vertex AI PALM
llm = VertexAI(
    model_name="text-bison@001",
    max_output_tokens=1024,
    temperature=0.2,
    top_p=0.8,
    top_k=40,
    verbose=True,
)

### Create SQL engine for BigQuery using SQLAlchemy

In [4]:
#define SQLAlchemy database engine

from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *
import pandas as pd

In [5]:
table_uri = f"bigquery://{project_id}/{dataset_id}"
engine = create_engine(f"bigquery://{project_id}/{dataset_id}")

In [7]:
#Testing all tables

query=f"""SELECT * FROM {project_id}.{dataset_id}.{table_name1}"""
engine.execute(query).first()

(31, 'Mickey Mouse', '1950-01-01', 'Jakarta', 'Indonesia', 'YES', 'Bachelor')

In [8]:
query=f"""SELECT * FROM {project_id}.{dataset_id}.{table_name2}"""
engine.execute(query).first()

('TX3008', 27, 'Surabaya', '2022-01-08', 'Eye Care', 800000, 'Ema')

In [9]:
query=f"""SELECT * FROM {project_id}.{dataset_id}.{table_name3}"""
engine.execute(query).first()

('TX3070', 61)

### Calling Langchain's SQLDatabase chain & Create LLM Prompt

In [10]:
# create SQLDatabase chain and prompt template
from langchain.prompts.prompt import PromptTemplate
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

def bq_qna(question):
  #create SQLDatabase instance from BQ engine
  db = SQLDatabase(engine=engine,metadata=MetaData(bind=engine),include_tables=[x for x in table_names])

  #create SQL DB Chain with the initialized LLM and above SQLDB instance
  db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, return_intermediate_steps=True)

  #Define prompt for BigQuery SQL generation
  _googlesql_prompt = """You are a GoogleSQL expert. Given an input question, first create a syntactically correct GoogleSQL query to run, then look at the results of the query and return the answer to the input question.
  Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per GoogleSQL. You can order the results to return the most informative data in the database.
  Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
  Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
  Use the following format:
  Question: "Question here"
  SQLQuery: "SQL Query to run"
  SQLResult: "Result of the SQLQuery"
  Answer: "Final answer here"
  Only use the following tables:
  {table_info}

  The join between tables:
  one to many join: clinic_dataset.customerdata.custid = clinic_dataset.ordertransaction.custid
  one to many join: clinic_dataset.ordertransaction.transactionid = clinic_dataset.satisfactionsurvey.transactionid
  
  If someone asks for aggregation on a STRING data type column, then CAST column as NUMERIC before you do the aggregation.

  If someone asks for specific month or date, always use start date and end date from transactiondate column.
  
  If someone asks for specific city or location of customer, use city_address column.
  
  If someone asks for specific city or location of transaction, use transactsitelocation column.

  If someone asks for column names in the table, use the following format:
  SELECT column_name
  FROM `{project_id}.{dataset_id}`.INFORMATION_SCHEMA.COLUMNS
  WHERE table_name in {table_info}

  Question: {input}"""

  GOOGLESQL_PROMPT = PromptTemplate(
      input_variables=["input", "table_info", "top_k", "project_id", "dataset_id"],
      template=_googlesql_prompt,
  )

  #passing question to the prompt template
  final_prompt = GOOGLESQL_PROMPT.format(input=question, project_id =project_id, dataset_id=dataset_id, table_info=table_names, top_k=10000)

  #pass final prompt to SQL Chain
  output = db_chain(final_prompt)


  return output['result'], output['intermediate_steps'][1]


### Testing the SQL Generation

In [11]:
#Testing sql generation 1
bq_qna('show me total order for each customer')



> Entering new SQLDatabaseChain chain...
You are a GoogleSQL expert. Given an input question, first create a syntactically correct GoogleSQL query to run, then look at the results of the query and return the answer to the input question.
  Unless the user specifies in the question a specific number of examples to obtain, query for at most 10000 results using the LIMIT clause as per GoogleSQL. You can order the results to return the most informative data in the database.
  Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
  Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
  Use the following format:
  Question: "Question here"
  SQLQuery: "SQL Query to run"
  SQLResult: "Result of the SQLQuery"
  Answer: "Fin

('[42, 51, 49, 48, 55]',
 'SELECT customerdata.custid, COUNT(ordertransaction.transactionid) AS total_order\nFROM customerdata\nJOIN ordertransaction ON customerdata.custid = ordertransaction.custid\nGROUP BY customerdata.custid\nORDER BY total_order DESC\nLIMIT 5')

In [12]:
#Testing sql generation
bq_qna('what is the highest services sold in April 2023')



> Entering new SQLDatabaseChain chain...
You are a GoogleSQL expert. Given an input question, first create a syntactically correct GoogleSQL query to run, then look at the results of the query and return the answer to the input question.
  Unless the user specifies in the question a specific number of examples to obtain, query for at most 10000 results using the LIMIT clause as per GoogleSQL. You can order the results to return the most informative data in the database.
  Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
  Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
  Use the following format:
  Question: "Question here"
  SQLQuery: "SQL Query to run"
  SQLResult: "Result of the SQLQuery"
  Answer: "Fin

('Covid Test',
 "SELECT servicecategory\nFROM `bqstackdemo.clinic_dataset.ordertransaction`\nWHERE transactiondate BETWEEN '2023-04-01' AND '2023-04-30'\nORDER BY transactionamount DESC\nLIMIT 1")

In [13]:
#Testing 3 - Metadata queries
bq_qna('what are the columns in customer table')



> Entering new SQLDatabaseChain chain...
You are a GoogleSQL expert. Given an input question, first create a syntactically correct GoogleSQL query to run, then look at the results of the query and return the answer to the input question.
  Unless the user specifies in the question a specific number of examples to obtain, query for at most 10000 results using the LIMIT clause as per GoogleSQL. You can order the results to return the most informative data in the database.
  Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
  Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
  Use the following format:
  Question: "Question here"
  SQLQuery: "SQL Query to run"
  SQLResult: "Result of the SQLQuery"
  Answer: "Fin

('Answer: custid, custname, dateofbirth, city_address, nationality, memberstatus, education',
 "SELECT column_name\nFROM `bqstackdemo.clinic_dataset`.INFORMATION_SCHEMA.COLUMNS\nWHERE table_name = 'customerdata'")

In [14]:
#Testing sql generation- Joins capability
bq_qna('what is the average survey score in each cities')



> Entering new SQLDatabaseChain chain...
You are a GoogleSQL expert. Given an input question, first create a syntactically correct GoogleSQL query to run, then look at the results of the query and return the answer to the input question.
  Unless the user specifies in the question a specific number of examples to obtain, query for at most 10000 results using the LIMIT clause as per GoogleSQL. You can order the results to return the most informative data in the database.
  Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
  Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
  Use the following format:
  Question: "Question here"
  SQLQuery: "SQL Query to run"
  SQLResult: "Result of the SQLQuery"
  Answer: "Fin

('Medan',
 'SELECT city_address, AVG(surveyscore) AS average_survey_score\nFROM clinic_dataset.customerdata AS c\nJOIN clinic_dataset.ordertransaction AS o ON c.custid = o.custid\nJOIN clinic_dataset.satisfactionsurvey AS s ON o.transactionid = s.transactionid\nGROUP BY city_address\nORDER BY average_survey_score DESC')

In [15]:
#Testing 4 - Joins & asking in bahasa
bq_qna('tampilkan rata rata hasil survey untuk tiap lokasi transaksi')



> Entering new SQLDatabaseChain chain...
You are a GoogleSQL expert. Given an input question, first create a syntactically correct GoogleSQL query to run, then look at the results of the query and return the answer to the input question.
  Unless the user specifies in the question a specific number of examples to obtain, query for at most 10000 results using the LIMIT clause as per GoogleSQL. You can order the results to return the most informative data in the database.
  Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
  Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
  Use the following format:
  Question: "Question here"
  SQLQuery: "SQL Query to run"
  SQLResult: "Result of the SQLQuery"
  Answer: "Fin

('Makassar, Bandung, Surabaya, Medan, Semarang, Jakarta',
 'SELECT transactsitelocation, AVG(surveyscore) AS avg_survey_score\nFROM `bqstackdemo.clinic_dataset.ordertransaction` AS t1\nJOIN `bqstackdemo.clinic_dataset.satisfactionsurvey` AS t2\nON t1.transactionid = t2.transactionid\nGROUP BY transactsitelocation\nORDER BY avg_survey_score DESC')

In [16]:
#Testing 4 - Joins & asking in bahasa
bq_qna('tampilkan rata rata hasil survey berdasarkan domisili pelanggan')



> Entering new SQLDatabaseChain chain...
You are a GoogleSQL expert. Given an input question, first create a syntactically correct GoogleSQL query to run, then look at the results of the query and return the answer to the input question.
  Unless the user specifies in the question a specific number of examples to obtain, query for at most 10000 results using the LIMIT clause as per GoogleSQL. You can order the results to return the most informative data in the database.
  Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
  Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
  Use the following format:
  Question: "Question here"
  SQLQuery: "SQL Query to run"
  SQLResult: "Result of the SQLQuery"
  Answer: "Fin

('Medan, Bandung, Manado, Bandar Lampung, Palembang',
 'SELECT city_address, AVG(surveyscore) AS average_survey_score\nFROM clinic_dataset.customerdata AS c\nJOIN clinic_dataset.ordertransaction AS o\nON c.custid = o.custid\nJOIN clinic_dataset.satisfactionsurvey AS s\nON o.transactionid = s.transactionid\nGROUP BY city_address\nORDER BY average_survey_score DESC\nLIMIT 5')